<a href="https://colab.research.google.com/github/OmriMan/EnsembleLearning_EnsembleDiversity_Assignment4/blob/main/Ensemble_Ass4_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/chazzer/ml-grandmaster-decision-tree-classifier

Packages

In [94]:
import numpy as np  
import pandas as pd  

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from matplotlib import pyplot as plt  
from sklearn.tree import plot_tree 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

import random
from tabulate import tabulate

#Dataset -Titanic

Source: https://www.kaggle.com/competitions/titanic/data

survival: Survival, 0 = No, 1 = Yes

pclass: Ticket class, 1 = 1st, 2 = 2nd, 3 = 3rd

sex: Sex. Female = 1,Male = 0

Age: Age in years.Missing values ​​were replaced by the mean ages in the data set

sibsp: numer of siblings / spouses aboard the Titanic

parch: number of parents / children aboard the Titanic

ticket: Ticket number.We did not use this feature

fare: Passenger fare.Missing values ​​were replaced by the mean fare values in the data set

cabin: Cabin number.We did not use this feature

embarked: Port of Embarkation C=Cherbourg,Q=Queenstown,S=Southampton.We replaced the values ​​as follows: S=0,C=1,Q=2

In [95]:
def preprocess(df):
    # Putting on index to each dataset before split it
    df = df.set_index("PassengerId")
    # Sex
    change = {'female':1,'male':0}
    df.Sex = df.Sex.map(change)

    # Embarked
    change = {'S':0,'C':1,'Q':2}
    df.Embarked = df.Embarked.map(change)
    df['Embarked'] = df['Embarked'].replace(to_replace = np.nan, value =1)

    # df['Age'] = df['Age'].groupby(df['Age'].isnull()).mean()
    df['Age'] = df['Age'].replace(to_replace = np.nan, value =df['Age'].mean())
    # df['Age'] = df['Age'].replace(to_replace = np.nan, value =50)
    df['Fare'] = df['Fare'].replace(to_replace = np.nan, value =df['Fare'].mean())
    df = df.drop(columns=['Name','Ticket','Cabin'])
    return df

Load the data set

In [96]:
df = pd.read_csv('/content/sample_data/titanic.csv')
df.isna().sum()

PassengerId       0
Survived          0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [97]:
df = preprocess(df)
features = ['Pclass',	'Sex',	'Age',	'SibSp',	'Parch',	'Fare',	'Embarked']
X = df.loc[:, features]
y = df.loc[:, ['Survived']]

Split the data into training and testin sets

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .75)
X_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1140,2,1,29.000000,1,0,26.0000,0.0
534,3,1,29.881138,0,2,22.3583,1.0
460,3,0,29.881138,0,0,7.7500,2.0
1151,3,0,21.000000,0,0,7.7750,0.0
394,1,1,23.000000,1,0,113.2750,1.0
...,...,...,...,...,...,...,...
365,3,0,29.881138,1,0,15.5000,2.0
495,3,0,21.000000,0,0,8.0500,0.0
709,1,1,22.000000,0,0,151.5500,0.0


#Target class distribution


In [99]:
target = df['Survived']
target.value_counts()

0    815
1    494
Name: Survived, dtype: int64

#Define and train the models with default hyper parameters
we are using 100 trees for each ensemble. Leave the other parameters at their default settings.


Random Forest

In [100]:
model_rf = RandomForestClassifier(n_estimators=100)

model_rf.fit(X_train,np.ravel(y_train))

RandomForestClassifier()

In [101]:
inputs = pd.DataFrame(X_test)
prediction_RandomForestClassifier = model_rf.predict(
    inputs
)
accuracy_RandomForestClassifier = accuracy_score(y_test,prediction_RandomForestClassifier)
print(f"Accuracy score of RandomForestClassifier = {accuracy_RandomForestClassifier}")

precision_RandomForestClassifier = precision_score(y_test,prediction_RandomForestClassifier,average='weighted')
print(f"Precision score of RandomForestClassifier = {precision_RandomForestClassifier}")

Accuracy score of RandomForestClassifier = 0.8597560975609756
Precision score of RandomForestClassifier = 0.8599780761852562


Extremely Randomized Trees

In [102]:
model_ExtraTreesClassifier = ExtraTreesClassifier(n_estimators=100)
model_ExtraTreesClassifier.fit(X_train,np.ravel(y_train))

ExtraTreesClassifier()

In [103]:
inputs = pd.DataFrame(X_test)
prediction_ExtraTreesClassifier = model_ExtraTreesClassifier.predict(
    inputs
)
accuracy_ExtraTreesClassifier = accuracy_score(y_test,prediction_ExtraTreesClassifier)
print(f"Accuracy score of ExtraTreesClassifier = {accuracy_ExtraTreesClassifier}")

precision_ExtraTreesClassifier = precision_score(y_test,prediction_ExtraTreesClassifier,average='weighted')
print(f"Precision score of ExtraTreesClassifier = {precision_ExtraTreesClassifier}")

Accuracy score of ExtraTreesClassifier = 0.8353658536585366
Precision score of ExtraTreesClassifier = 0.8359602929163262


#Summary - Titanic dataset

In [104]:
print(tabulate([["Accuracy",accuracy_RandomForestClassifier,accuracy_ExtraTreesClassifier],["Precision",precision_RandomForestClassifier,precision_ExtraTreesClassifier]], headers=["Metric\Model","RandomForestClassifier","ExtraTreesClassifier"], tablefmt="grid"))

+----------------+--------------------------+------------------------+
| Metric\Model   |   RandomForestClassifier |   ExtraTreesClassifier |
+================+==========================+========================+
| Accuracy       |                 0.859756 |               0.835366 |
+----------------+--------------------------+------------------------+
| Precision      |                 0.859978 |               0.83596  |
+----------------+--------------------------+------------------------+


##Diversity measurements

In [105]:
# import diversity_measures as diversity_measures
from diversity_measures import *

Double Fault Measure

In [106]:
inputs_train = pd.DataFrame(X_train)
def DoubleFaultMeasure(s, set_of_classifiers):
  train_set=s[0]
  sum = 0
  i_index = 0
  predictions =[]
  # For each classifier, insert all its predictions (depending on the index of x) into the list .
  for classifier in set_of_classifiers:
    preds = classifier.predict(train_set.values)
    predictions.append((classifier,preds))
  for i,model_i_predictions in predictions:
    # i.fit(X_train,np.ravel(y_train))
    # preds_i = i.predict(inputs_train.values)
    for j,model_j_predictions in predictions[i_index:]:
      num_of_instances = 0
      if i != j: # if not the same tree
        # preds_j = j.predict(inputs_train.values)
        for k in range(len(np.ravel(y_train))):
          if (model_i_predictions[k] != np.ravel(y_train)[k] and model_j_predictions[k] != np.ravel(y_train)[k]): # if both trees misclasified then add 1 to num of instances...
            num_of_instances += 1

        sum += num_of_instances
    i_index += 1
    # print(i_index)

  res = (2*sum)/(len(train_set)*len(set_of_classifiers)*(len(set_of_classifiers)-1))
  return res

In [107]:
# DoubleFaultMeasure((X_train,y_train),model_rf)

Kohavi Wolpert

In [108]:
def kohavi_wolpert(s,m):
  '''
  s - the original training set tuple of (X_train,y_train) --> ((dataframe),((dataframe))
  s[0] = x train
  s[1] = y train
  m - list of classifiers examined
  '''
  x = s[0]
  y=s[1]
  predictions =[]
  sum =0
  # For each classifier, insert all its predictions (depending on the index of x) into the list .
  for classifier in m:
    preds = classifier.predict(x.values)
    predictions.append(preds)
  # Run through all the records in the training set and for each record : count the number of classifiers that made an incorrect classification(misclassified)
  l_i=0
  for (index_x, x_i),(index_y, y_i),pred_index in zip(x.iterrows(),y.iterrows(),range(len(predictions[0]))):
    l_i=0
    for model_i_prediction in predictions:
      # l_i = number of classifiers misclassified x_i,y_i
      if model_i_prediction[pred_index] != y_i.values:
        l_i+=1
      #end sum all misclassification for x_i,y_i
    #sum = sum + l_i * (len(m) - l_i)
    sum = sum + ( l_i * ( len(m)-l_i ) ) 
  # print(f"sum/(len(x)*(len(m)**2)) : {sum/(len(x)*(len(m)**2))}")
  return (sum) / ( len(y) * (len(m)**2) )

In [109]:
# kohavi_wolpert((X_train,y_train),model_rf.estimators_)

Inter Rater Measure

In [110]:
def inter_rater_measure(s,m):
    '''
    s - the original training set tuple of (X_train,y_train) --> ((dataframe),((dataframe))
    s[0] = x train
    s[1] = y train
    m - list of classifiers examined
    '''
    x = s[0]
    y=s[1]
    predictions =[]
    sum =0
    # For each classifier, insert all its predictions (depending on the index of x) into the list .
    for classifier in m:
        preds = classifier.predict(x.values)
        predictions.append(preds)
    # Run through all the records in the training set and for each record : count the number of classifiers that made an incorrect classification(misclassified)
    l=[]
    for (index_x, x_i),(index_y, y_i),pred_index in zip(x.iterrows(),y.iterrows(),range(len(predictions[0]))):
        l_i=0
        for model_i_prediction in predictions:
            # l_i = number of classifiers misclassified x_i,y_i
            if model_i_prediction[pred_index] != y_i.values:
                l_i+=1
            #end sum all misclassification for x_i,y_i
        l.append(l_i)
        #sum = sum + l_i * (len(m) - l_i)
        sum = sum + ( l_i * ( len(m) - l_i) )
    # p = 1 - ( sum(l) / (|s| *|m|) )
    sum_l=0
    for val in l:
        sum_l+=val
    p = 1 - ( sum_l / (len(x)*len(m)) )
    if p==1:
        return 1
    result = 1 - (sum / ( len(x)*len(m)*(len(m)-1)*p*(1-p)  ) )
    return result

In [111]:
# inter_rater_measure((X_train,y_train),model_rf.estimators_)

General Diversity

In [112]:
import statistics
def general_diversity(s,m):
  '''
  s - the original training set tuple of (X_train,y_train) --> ((dataframe),((dataframe))
  s[0] = x train
  s[1] = y train
  m - list of classifiers examined
  '''
  x = s[0]
  y=s[1]
  predictions =[]
  # For each classifier, insert all its predictions (depending on the index of x) into the list .
  for classifier in m:
    preds = classifier.predict(x.values)
    predictions.append(preds)
  # Run through all the records in the training set and for each record : count the number of classifiers that made an incorrect classification(misclassified)
  v=[]
  for (index_x, x_i),(index_y, y_i),pred_index in zip(x.iterrows(),y.iterrows(),range(len(predictions[0]))):
    l_i=0
    for model_i_prediction in predictions:
      # l_i = number of classifiers misclassified x_i,y_i
      if model_i_prediction[pred_index] != y_i.values:
        l_i+=1
      #end sum all misclassification for x_i,y_i
    v_i= (len(m)-l_i) / len(m)
    v.append(v_i)
  # end of run through all the records in the training set and for each record : count the number of classifiers that made an incorrect classification(misclassified)
  # compute variance of v using statistics.variance
  variance_v = statistics.variance(v)
  return variance_v

In [113]:
# general_diversity((X_train,y_train),model_rf.estimators_)

#Bagging using diversity

In [114]:
from sklearn.metrics import classification_report, confusion_matrix
from functools import reduce

def bagging_using_Diversity(i,t,s,d):
  '''
  i - a base inducer
  t - number of iterations
  s - the original training set
  d- diversity measure - { ""  :kohavi_wolpert , "" :inter_rater_measure ,"general diversity": general_diversity }
  '''
  diversity_measure = {"Double Fault Measure": DoubleFaultMeasure, "kohavi wolpert"  :kohavi_wolpert , "inter rater measure" :inter_rater_measure ,"general diversity": general_diversity }
  m_ts =[]
  #m_ts - list of tuples - each tuple in the following format : (model,number_of_misclassifieds)
  # m_ts --> [(model1,number_of_misclassifieds1),(model2,number_of_misclassifieds2),...,(model_t,number_of_misclassifieds_t)]
  for iter in range(t):
    s_tag = s.sample(frac = 1)
    m_t = i.estimators_[iter]
    x = s_tag.loc[:, features]
    y = s_tag.loc[:, ['Survived']]
    preds = m_t.predict(x.values)
    # confusion_matrix(y,preds) - [[ture_positive, false_positive][false_neg,true_neg]]
    number_of_misclassifieds = confusion_matrix(y,preds)[0][1] + confusion_matrix(y,preds)[1][0]
    m_ts.append((m_t,number_of_misclassifieds))
  # end of rows 1-3
  x = s.loc[:, features]
  y = s.loc[:, ['Survived']]
  # 5
  m_tag=[]
  # use reduce() and min() to find the minimum value in the second element of each tuple
  min_misclassifields = reduce(lambda x, y: min(x, y[1]), m_ts, float("inf"))
  # Find an element (model,number_of_misclassifieds) in m_ts(list of tuples).
  m_tags_optinals = [item for item in m_ts if item[1] == min_misclassifields]
  # m_tags_optinals - list of tuples [(model1,min_misclassifields),...]
  m_tag.append(m_tags_optinals[0][0])
  m_ts.remove(m_tags_optinals[0])
  # 6
  for i in range(1,int((t/2)-1)):
    results_of_d =[]
    # results_of_d - list of tuples - each tuple in the following format : (model,diversity measure value)
    if (d=="Double Fault Measure" or d=="kohavi wolpert" or d=="general diversity"):
      # need to take the model that return maximum diversity measure
      print(i)
      for temp_model,number_of_misclassifieds in m_ts:
        # m_tag_only_models = [tup[0] for tup in m_tag]
        results_of_d.append( (temp_model, diversity_measure[d]((x,y),m_tag+[temp_model])) )
      # need to find maximum value of diversity measure
      # use reduce() and min() to find the minimum value in the second element of each tuple
      # results_of_d[0] = (model,diversity measure value)
      max_d = reduce(lambda x, y: max(x, y[1]), results_of_d, float("-inf"))
      # Find an element (model,diversity measure value) in results_of_d(list of tuples).
      m_tags_optinals = [item for item in  results_of_d if item[1] == max_d]
      # m_tags_optinals[0] = (model,diversity measure value), e.g: (DecisionTreeClassifier(max_features='auto', random_state=1913246406), 0.014287578288100209)
      # m_tags_optinals[0][0] - temp_model
      # m_tags_optinals[0][1] - maximum diversity measure value , and its for m_tag with temp_model
      # we want to add only the model to m_tag so we add m_tags_optinals[0][0]
      m_tag.append(m_tags_optinals[0][0])
      # need to find m_tags_optinals[0][0] (the model) in m_ts and remove it from m_ts
      model_num_of_mis_tuple_to_remove_from_m_ts = [item for item in m_ts if item[0] == m_tags_optinals[0][0]]
      m_ts.remove(model_num_of_mis_tuple_to_remove_from_m_ts[0])
    else:
      # need to take the model that return minimum diversity measure
      print(i)
      for temp_model,number_of_misclassifieds in m_ts:
        # m_tag_only_models = [tup[0] for tup in m_tag]
        results_of_d.append( (temp_model, diversity_measure[d]((x,y),m_tag+[temp_model])) )
      # need to find maximum value of diversity measure
      # use reduce() and min() to find the minimum value in the second element of each tuple
      # results_of_d[0] = (model,diversity measure value)
      min_d = reduce(lambda x, y: min(x, y[1]), results_of_d, float("inf"))
      # Find an element (model,diversity measure value) in results_of_d(list of tuples).
      m_tags_optinals = [item for item in  results_of_d if item[1] == min_d]
      # m_tags_optinals[0] = (model,diversity measure value), e.g: (DecisionTreeClassifier(max_features='auto', random_state=1913246406), 0.014287578288100209)
      # m_tags_optinals[0][0] - temp_model
      # m_tags_optinals[0][1] - minimum diversity measure value , and its for m_tag with temp_model
      # we want to add only the model to m_tag so we add m_tags_optinals[0][0]
      m_tag.append(m_tags_optinals[0][0])
      # need to find m_tags_optinals[0][0] (the model) in m_ts and remove it from m_ts
      model_num_of_mis_tuple_to_remove_from_m_ts = [item for item in m_ts if item[0] == m_tags_optinals[0][0]]
      m_ts.remove(model_num_of_mis_tuple_to_remove_from_m_ts[0])

  return m_tag

In [115]:
from copy import deepcopy

In [116]:
train_set_x_and_y = deepcopy(X_train.loc[:, features])
train_set_x_and_y['Survived'] = deepcopy(y_train['Survived'])

#Base inducer - Random Forest

In [117]:
RandomForestClassifier_bagging_using_Diversity_Double_Fault_Measure = deepcopy(model_rf)

In [118]:
RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert = deepcopy(model_rf)

In [119]:
RandomForestClassifier_bagging_using_Diversity_inter_rater_measure = deepcopy(model_rf)

In [120]:
RandomForestClassifier_bagging_using_Diversity_general_diversity = deepcopy(model_rf)

Kohavi Wolpert

In [121]:
models_RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert = bagging_using_Diversity(RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert,RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert.n_estimators,train_set_x_and_y,"kohavi wolpert")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [122]:
RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert.estimators_ = models_RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert
print("Random Forest Classifier - bagging using diversity kohavi-wolpert")
inputs = pd.DataFrame(X_test)
prediction_RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert = RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert.predict(inputs)
accuracy_RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert = accuracy_score(y_test,prediction_RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert)
print(f"Accuracy score of RandomForestClassifier using diversity kohavi-wolpert = {accuracy_RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert}")

precision_RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert = precision_score(y_test,prediction_RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert,average='weighted')
print(f"Precision score of RandomForestClassifier using diversity kohavi-wolpert = {precision_RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert}")

Random Forest Classifier - bagging using diversity kohavi-wolpert
Accuracy score of RandomForestClassifier using diversity kohavi-wolpert = 0.8475609756097561
Precision score of RandomForestClassifier using diversity kohavi-wolpert = 0.8477957431259705


Inter Rater Measure

In [123]:
models_RandomForestClassifier_bagging_using_Diversity_inter_rater_measure = bagging_using_Diversity(RandomForestClassifier_bagging_using_Diversity_inter_rater_measure,RandomForestClassifier_bagging_using_Diversity_inter_rater_measure.n_estimators,train_set_x_and_y,"inter rater measure")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [124]:
RandomForestClassifier_bagging_using_Diversity_inter_rater_measure.estimators_ = models_RandomForestClassifier_bagging_using_Diversity_inter_rater_measure
print("Random Forest Classifier - bagging using diversity Inter-Rater-Measure")
inputs = pd.DataFrame(X_test)
prediction_RandomForestClassifier_bagging_using_Diversity_inter_rater_measure = RandomForestClassifier_bagging_using_Diversity_inter_rater_measure.predict(inputs)
accuracy_RandomForestClassifier_bagging_using_Diversity_inter_rater_measure = accuracy_score(y_test,prediction_RandomForestClassifier_bagging_using_Diversity_inter_rater_measure)
print(f"Accuracy score of RandomForestClassifier using diversity Inter-Rater-Measure = {accuracy_RandomForestClassifier_bagging_using_Diversity_inter_rater_measure}")

precision_RandomForestClassifier_bagging_using_Diversity_inter_rater_measure = precision_score(y_test,prediction_RandomForestClassifier_bagging_using_Diversity_inter_rater_measure,average='weighted')
print(f"Precision score of RandomForestClassifier using diversity Inter-Rater-Measure = {precision_RandomForestClassifier_bagging_using_Diversity_inter_rater_measure}")

Random Forest Classifier - bagging using diversity Inter-Rater-Measure
Accuracy score of RandomForestClassifier using diversity Inter-Rater-Measure = 0.8536585365853658
Precision score of RandomForestClassifier using diversity Inter-Rater-Measure = 0.8542201540436457


General Diversity

In [125]:
models_RandomForestClassifier_bagging_using_Diversity_general_diversity = bagging_using_Diversity(RandomForestClassifier_bagging_using_Diversity_general_diversity,RandomForestClassifier_bagging_using_Diversity_general_diversity.n_estimators,train_set_x_and_y,"general diversity")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [126]:
RandomForestClassifier_bagging_using_Diversity_general_diversity.estimators_ = models_RandomForestClassifier_bagging_using_Diversity_general_diversity
print("Random Forest Classifier - bagging using diversity General Diversity")
inputs = pd.DataFrame(X_test)
prediction_RandomForestClassifier_bagging_using_Diversity_general_diversity = RandomForestClassifier_bagging_using_Diversity_general_diversity.predict(inputs)
accuracy_RandomForestClassifier_bagging_using_Diversity_general_diversity = accuracy_score(y_test,prediction_RandomForestClassifier_bagging_using_Diversity_general_diversity)
print(f"Accuracy score of RandomForestClassifier using diversity General Diversity = {accuracy_RandomForestClassifier_bagging_using_Diversity_general_diversity}")

precision_RandomForestClassifier_bagging_using_Diversity_general_diversity = precision_score(y_test,prediction_RandomForestClassifier_bagging_using_Diversity_general_diversity,average='weighted')
print(f"Precision score of RandomForestClassifier using diversity General Diversity = {precision_RandomForestClassifier_bagging_using_Diversity_general_diversity}")

Random Forest Classifier - bagging using diversity General Diversity
Accuracy score of RandomForestClassifier using diversity General Diversity = 0.8628048780487805
Precision score of RandomForestClassifier using diversity General Diversity = 0.8627360120709111


Double Fault Measure

In [127]:
# models_RandomForestClassifier_bagging_using_Diversity_Double_Fault_Measure = bagging_using_Diversity(RandomForestClassifier_bagging_using_Diversity_Double_Fault_Measure,RandomForestClassifier_bagging_using_Diversity_Double_Fault_Measure.n_estimators,train_set_x_and_y,"Double Fault Measure")

In [128]:
# RandomForestClassifier_bagging_using_Diversity_Double_Fault_Measure.estimators_ = models_RandomForestClassifier_bagging_using_Diversity_Double_Fault_Measure
# print("Random Forest Classifier - bagging using diversity Double Fault Measure")
# inputs = pd.DataFrame(X_test)
# prediction_RandomForestClassifier_bagging_using_Diversity_Double_Fault_Measure = RandomForestClassifier_bagging_using_Diversity_Double_Fault_Measure.predict(inputs)
# accuracy_RandomForestClassifier_bagging_using_Diversity_Double_Fault_Measure = accuracy_score(y_test,prediction_RandomForestClassifier_bagging_using_Diversity_Double_Fault_Measure)
# print(f"Accuracy score of RandomForestClassifier using diversity Double Fault Measure = {accuracy_RandomForestClassifier_bagging_using_Diversity_Double_Fault_Measure}")

# precision_RandomForestClassifier_bagging_using_Diversity_Double_Fault_Measure = precision_score(y_test,prediction_RandomForestClassifier_bagging_using_Diversity_Double_Fault_Measure,average='weighted')
# print(f"Precision score of RandomForestClassifier using diversity Double Fault Measure = {precision_RandomForestClassifier_bagging_using_Diversity_Double_Fault_Measure}")

#Summary - Base inducer - Random Forest

In [129]:
print(tabulate([["Accuracy",accuracy_RandomForestClassifier,
                 accuracy_RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert,
                 accuracy_RandomForestClassifier_bagging_using_Diversity_inter_rater_measure,
                 accuracy_RandomForestClassifier_bagging_using_Diversity_general_diversity],
                ["Precision",precision_RandomForestClassifier,
                 precision_RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert,
                 precision_RandomForestClassifier_bagging_using_Diversity_inter_rater_measure,
                 precision_RandomForestClassifier_bagging_using_Diversity_general_diversity]],
               headers=["Metric\Model","RandomForestClassifier",
                        "Random Forest-\nbagging using diversity\nkohavi-wolpert",
                        "Random Forest-\nbagging using diversity\nInter-Rater-Measure",
                        "Random Forest-\nbagging using diversity\nGeneral Diversity"], tablefmt="grid"))

+----------------+--------------------------+---------------------------+---------------------------+---------------------------+
| Metric\Model   |   RandomForestClassifier |            Random Forest- |            Random Forest- |            Random Forest- |
|                |                          |   bagging using diversity |   bagging using diversity |   bagging using diversity |
|                |                          |            kohavi-wolpert |       Inter-Rater-Measure |         General Diversity |
+================+==========================+===========================+===========================+===========================+
| Accuracy       |                 0.859756 |                  0.847561 |                  0.853659 |                  0.862805 |
+----------------+--------------------------+---------------------------+---------------------------+---------------------------+
| Precision      |                 0.859978 |                  0.847796 |                 

In [130]:
# print(tabulate([["Accuracy",accuracy_RandomForestClassifier,
#                  accuracy_RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert,
#                  accuracy_RandomForestClassifier_bagging_using_Diversity_inter_rater_measure,
#                  accuracy_RandomForestClassifier_bagging_using_Diversity_general_diversity,
#                  accuracy_RandomForestClassifier_bagging_using_Diversity_Double_Fault_Measure],
#                 ["Precision",precision_RandomForestClassifier,
#                  precision_RandomForestClassifier_bagging_using_Diversity_kohavi_wolpert,
#                  precision_RandomForestClassifier_bagging_using_Diversity_inter_rater_measure,
#                  precision_RandomForestClassifier_bagging_using_Diversity_general_diversity,
#                  precision_RandomForestClassifier_bagging_using_Diversity_Double_Fault_Measure]],
#                headers=["Metric\Model","RandomForestClassifier",
#                         "Random Forest-\nbagging using diversity\nkohavi-wolpert",
#                         "Random Forest-\nbagging using diversity\nInter-Rater-Measure",
#                         "Random Forest-\nbagging using diversity\nGeneral Diversity",
#                         "Random Forest-\nbagging using diversity\nDouble Fault Measure"], tablefmt="grid"))

#Base inducer - Extra Trees

In [131]:
ExtraTrees_Double_Fault = deepcopy(model_ExtraTreesClassifier)

In [132]:
ExtraTrees_kohavi_wolpert = deepcopy(model_ExtraTreesClassifier)

In [133]:
ExtraTrees_inter_rater = deepcopy(model_ExtraTreesClassifier)

In [134]:
ExtraTrees_general_diversity = deepcopy(model_ExtraTreesClassifier)

In [135]:
train_set_x_and_y = deepcopy(X_train.loc[:, features])
train_set_x_and_y['Survived'] = deepcopy(y_train['Survived'])

Kohavi Wolpert

In [136]:
models_ExtraTrees_kohavi_wolpert = bagging_using_Diversity(ExtraTrees_kohavi_wolpert,ExtraTrees_kohavi_wolpert.n_estimators,train_set_x_and_y,"kohavi wolpert")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [137]:
ExtraTrees_kohavi_wolpert.estimators_ = models_ExtraTrees_kohavi_wolpert
print("Extra Trees Classifier - bagging using diversity kohavi-wolpert")
inputs = pd.DataFrame(X_test)
prediction_ExtraTrees_kohavi_wolpert = ExtraTrees_kohavi_wolpert.predict(inputs)
accuracy_ExtraTrees_kohavi_wolpert = accuracy_score(y_test,prediction_ExtraTrees_kohavi_wolpert)
print(f"Accuracy score of Extra Trees using diversity kohavi-wolpert = {accuracy_ExtraTrees_kohavi_wolpert}")

precision_ExtraTrees_kohavi_wolpert = precision_score(y_test,prediction_ExtraTrees_kohavi_wolpert,average='weighted')
print(f"Precision score of Extra Trees using diversity kohavi-wolpert = {precision_ExtraTrees_kohavi_wolpert}")

Extra Trees Classifier - bagging using diversity kohavi-wolpert
Accuracy score of Extra Trees using diversity kohavi-wolpert = 0.8384146341463414
Precision score of Extra Trees using diversity kohavi-wolpert = 0.8392134089408121


Inter Rater measure

In [138]:
models_ExtraTrees_inter_rater = bagging_using_Diversity(ExtraTrees_inter_rater,ExtraTrees_inter_rater.n_estimators,train_set_x_and_y,"inter rater measure")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [139]:
ExtraTrees_inter_rater.estimators_ = models_ExtraTrees_inter_rater
print("Extra Trees Classifier - bagging using diversity inter-rater")
inputs = pd.DataFrame(X_test)
prediction_ExtraTrees_inter_rater = ExtraTrees_inter_rater.predict(inputs)
accuracy_ExtraTrees_inter_rater = accuracy_score(y_test,prediction_ExtraTrees_inter_rater)
print(f"Accuracy score of Extra Trees using diversity inter-rater = {accuracy_ExtraTrees_inter_rater}")

precision_ExtraTrees_inter_rater = precision_score(y_test,prediction_ExtraTrees_inter_rater,average='weighted')
print(f"Precision score of Extra Trees using diversity inter-rater = {precision_ExtraTrees_inter_rater}")

Extra Trees Classifier - bagging using diversity inter-rater
Accuracy score of Extra Trees using diversity inter-rater = 0.8384146341463414
Precision score of Extra Trees using diversity inter-rater = 0.8392134089408121


General Diversity

In [140]:
models_ExtraTrees_general_diversity = bagging_using_Diversity(ExtraTrees_general_diversity,ExtraTrees_general_diversity.n_estimators,train_set_x_and_y,"general diversity")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [141]:
ExtraTrees_general_diversity.estimators_ = models_ExtraTrees_general_diversity
print("Extra Trees Classifier - bagging using diversity general diversity")
inputs = pd.DataFrame(X_test)
prediction_ExtraTrees_general_diversity = ExtraTrees_general_diversity.predict(inputs)
accuracy_ExtraTrees_general_diversity = accuracy_score(y_test,prediction_ExtraTrees_general_diversity)
print(f"Accuracy score of Extra Trees using diversity general diversity = {accuracy_ExtraTrees_general_diversity}")

precision_ExtraTrees_general_diversity = precision_score(y_test,prediction_ExtraTrees_general_diversity,average='weighted')
print(f"Precision score of Extra Trees using diversity general diversity = {precision_ExtraTrees_general_diversity}")

Extra Trees Classifier - bagging using diversity general diversity
Accuracy score of Extra Trees using diversity general diversity = 0.8384146341463414
Precision score of Extra Trees using diversity general diversity = 0.8392134089408121


Double Fault

In [142]:
# models_ExtraTrees_Double_Fault = bagging_using_Diversity(ExtraTrees_Double_Fault,ExtraTrees_Double_Fault.n_estimators,train_set_x_and_y,""Double Fault Measure")

In [143]:
# ExtraTrees_Double_Fault.estimators_ = models_ExtraTrees_Double_Fault
# print("Extra Trees Classifier - bagging using diversity Double Fault")
# inputs = pd.DataFrame(X_test)
# prediction_ExtraTrees_Double_Fault = ExtraTrees_Double_Fault.predict(inputs)
# accuracy_ExtraTrees_Double_Fault = accuracy_score(y_test,prediction_ExtraTrees_Double_Fault)
# print(f"Accuracy score of Extra Trees using diversity Double Fault = {accuracy_ExtraTrees_Double_Fault}")

# precision_ExtraTrees_Double_Fault = precision_score(y_test,prediction_ExtraTrees_Double_Fault,average='weighted')
# print(f"Precision score of Extra Trees using diversity Double Fault = {precision_ExtraTrees_Double_Fault}")

#Summary - Base inducer - Extra Trees

In [144]:
print(tabulate([["Accuracy",accuracy_ExtraTreesClassifier,
                 accuracy_ExtraTrees_kohavi_wolpert,
                 accuracy_ExtraTrees_inter_rater,
                 accuracy_ExtraTrees_general_diversity],
                ["Precision",precision_ExtraTreesClassifier,
                 precision_ExtraTrees_kohavi_wolpert,
                 precision_ExtraTrees_inter_rater,
                 precision_ExtraTrees_general_diversity]],
               headers=["Metric\Model","ExtraTreesClassifier",
                        "Extra Trees-\nbagging using diversity\nkohavi-wolpert",
                        "Extra Trees-\nbagging using diversity\nInter-Rater-Measure",
                        "Extra Trees-\nbagging using diversity\nGeneral Diversity"], tablefmt="grid"))

+----------------+------------------------+---------------------------+---------------------------+---------------------------+
| Metric\Model   |   ExtraTreesClassifier |              Extra Trees- |              Extra Trees- |              Extra Trees- |
|                |                        |   bagging using diversity |   bagging using diversity |   bagging using diversity |
|                |                        |            kohavi-wolpert |       Inter-Rater-Measure |         General Diversity |
+================+========================+===========================+===========================+===========================+
| Accuracy       |               0.835366 |                  0.838415 |                  0.838415 |                  0.838415 |
+----------------+------------------------+---------------------------+---------------------------+---------------------------+
| Precision      |               0.83596  |                  0.839213 |                  0.839213 |     

In [145]:
# print(tabulate([["Accuracy",accuracy_ExtraTreesClassifier,
#                  accuracy_ExtraTrees_kohavi_wolpert,
#                  accuracy_ExtraTrees_inter_rater,
#                  accuracy_ExtraTrees_general_diversity,
#                  accuracy_ExtraTrees_Double_Fault],
#                 ["Precision",precision_ExtraTreesClassifier,
#                  precision_ExtraTrees_kohavi_wolpert,
#                  precision_ExtraTrees_inter_rater,
#                  precision_ExtraTrees_general_diversity,
#                  precision_ExtraTrees_Double_Fault]],
#                headers=["Metric\Model","ExtraTreesClassifier",
#                         "Extra Trees-\nbagging using diversity\nkohavi-wolpert",
#                         "Extra Trees-\nbagging using diversity\nInter-Rater-Measure",
#                         "Extra Trees-\nbagging using diversity\nGeneral Diversity",
#                         "Extra Trees-\nbagging using diversity\nDouble Fault Measure"], tablefmt="grid"))